# Telecom X - Análisis de Evasión de Clientes (LATAM)

La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.

**El análisis contiene:**
- Importación y manipulación de datos desde una API
- ETL (Extracción, Transformación y Carga)
- Visualizaciones estratégicas para identificar patrones y tendencias
- EDA e informe con insights relevantes


##  Preparación (Imports y Configuración)


In [ ]:
import pandas as pd
import numpy as np
import requests, json
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (6,4)


##  Extracción — Carga desde la API (JSON)


In [ ]:
# API
URL = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"

# Accediendo a la API
resp = requests.get(URL)
resp.raise_for_status()

# Recuperando los resultados
resultado = json.loads(resp.text)

# Obteniendo vista previa del DataFrame base (anidado)
df = pd.DataFrame(resultado)
display(df.head())
df.info()


##  Transformación — Normalización de columnas anidadas


In [ ]:
# Mantener id y objetivo
id_churn = df[['customerID', 'Churn']].copy()

# Normalización de estructuras anidadas
customer = pd.json_normalize(df['customer'])
phone    = pd.json_normalize(df['phone'])
internet = pd.json_normalize(df['internet'])
account  = pd.json_normalize(df['account'])

# Concatenación para crear el DataFrame analítico
telecom_df = pd.concat([id_churn, customer, phone, internet, account], axis=1)
display(telecom_df.sample(4))


##  Estructura — Renombrado de columnas y verificación


In [ ]:
telecom_df.rename(columns={
    'customerID': 'id_cliente',
    'Churn': 'abandono',
    'gender': 'genero',
    'SeniorCitizen': 'es_ciudadano_senior',
    'Partner': 'tiene_pareja',
    'Dependents': 'tiene_dependientes',
    'tenure': 'meses_en_empresa',
    'PhoneService': 'servicio_telefonico',
    'MultipleLines': 'lineas_multiples',
    'InternetService': 'tipo_internet',
    'OnlineSecurity': 'seguridad_online',
    'OnlineBackup': 'respaldo_online',
    'DeviceProtection': 'proteccion_dispositivo',
    'TechSupport': 'soporte_tecnico',
    'StreamingTV': 'tv_streaming',
    'StreamingMovies': 'peliculas_streaming',
    'Contract': 'tipo_contrato',
    'PaperlessBilling': 'factura_electronica',
    'PaymentMethod': 'metodo_pago',
    'Charges.Monthly': 'cargos_mensuales',
    'Charges.Total': 'cargos_totales'
}, inplace=True)

display(telecom_df.head())
telecom_df.info()


##  Control de calidad (nulos, duplicados, formatos)


In [ ]:
print("Duplicados exactos:", telecom_df.duplicated().sum())
display(telecom_df.isnull().sum().sort_values(ascending=False))


##  Limpieza de texto (IDs, contrato, método de pago)


In [ ]:
# ID
telecom_df['id_cliente'] = telecom_df['id_cliente'].astype(str).str.strip()

# Contrato
telecom_df['tipo_contrato'] = (telecom_df['tipo_contrato']
                               .astype(str).str.lower()
                               .str.replace('-', ' ', regex=False)
                               .str.replace('_', ' ', regex=False)
                               .str.strip())

# Método de pago
telecom_df['metodo_pago'] = (telecom_df['metodo_pago']
                             .astype(str).str.lower()
                             .str.replace('\n', ' ', regex=True)
                             .str.strip())

display(telecom_df[['id_cliente','tipo_contrato','metodo_pago']].sample(5))


##  Limpieza numérica (conversiones y nulos críticos)


In [ ]:
# Convertir a numérico y tratar nulos
telecom_df['cargos_mensuales'] = pd.to_numeric(telecom_df['cargos_mensuales'], errors='coerce')
telecom_df['cargos_totales']   = pd.to_numeric(telecom_df['cargos_totales'].replace(' ', np.nan), errors='coerce')
telecom_df['meses_en_empresa'] = pd.to_numeric(telecom_df['meses_en_empresa'], errors='coerce', downcast='integer')

print("Nulos en cargos_totales:", telecom_df['cargos_totales'].isna().sum())
telecom_df = telecom_df.dropna(subset=['cargos_totales'])  # suelen ser clientes muy nuevos o datos incompletos
print("Tamaño tras limpiar:", telecom_df.shape)


##  Estandarización (Yes/No → 1/0) y nueva variable `cuentas_diarias`


In [ ]:
# Binarias frecuentes
col_binario = ['abandono','tiene_pareja','tiene_dependientes','servicio_telefonico','factura_electronica']
for c in col_binario:
    if c in telecom_df.columns:
        telecom_df[c] = telecom_df[c].astype(str).str.strip().str.title()

telecom_df[col_binario] = (telecom_df[col_binario]
                           .replace({'Yes':1,'No':0,'':np.nan})
                           .fillna(0)
                           .astype(int))

# Nueva columna: prorrateo a valor diario
telecom_df['cuentas_diarias'] = telecom_df['cargos_mensuales'] / 30.0

display(telecom_df[['cargos_mensuales','cuentas_diarias']].sample(5))
telecom_df.info()


##  Análisis descriptivo (numéricas)


In [ ]:
desc = telecom_df[['meses_en_empresa','cargos_mensuales','cargos_totales','cuentas_diarias']].describe()
display(desc)

skew = telecom_df[['meses_en_empresa','cargos_mensuales','cargos_totales','cuentas_diarias']].skew()
kurt = telecom_df[['meses_en_empresa','cargos_mensuales','cargos_totales','cuentas_diarias']].kurt()
display(pd.DataFrame({'skew':skew,'kurtosis':kurt}))


##  Distribución de la variable de evasión (`abandono`)


In [ ]:
abandono_counts = telecom_df['abandono'].value_counts().sort_index()
labels = ['Permanece (0)', 'Abandona (1)']

# Barras
plt.figure()
plt.bar(labels, abandono_counts.values)
plt.title('Distribución de Clientes según Abandono')
plt.ylabel('Número de Clientes')
for i,v in enumerate(abandono_counts.values):
    plt.text(i, v*0.98, f'{v}', ha='center', va='top')
plt.show()

# Pastel
plt.figure()
plt.pie(abandono_counts.values, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Proporción de Clientes según Abandono')
plt.show()


##  Churn por variables categóricas (contrato, pago, internet, género)


In [ ]:
def churn_por_categoria(df, col):
    tab = pd.crosstab(df[col], df['abandono'], normalize='index') * 100
    tab.columns = ['% Permanece (0)','% Abandona (1)']
    display(tab.sort_values('% Abandona (1)', ascending=False))
    ax = tab.plot(kind='bar', stacked=True, figsize=(7,4))
    ax.set_title(f'Abandono por {col}')
    ax.set_ylabel('Porcentaje')
    ax.set_xlabel('')
    plt.tight_layout()
    plt.show()

for c in ['tipo_contrato','metodo_pago','tipo_internet','genero']:
    if c in telecom_df.columns:
        churn_por_categoria(telecom_df, c)


##  Churn por rangos (bins) de variables numéricas


In [ ]:
def churn_por_bins(df, col, q=5):
    df_tmp = df.copy()
    df_tmp[f'{col}_bin'] = pd.qcut(df_tmp[col], q=q, duplicates='drop')
    tab = pd.crosstab(df_tmp[f'{col}_bin'], df_tmp['abandono'], normalize='index') * 100
    tab.columns = ['% Permanece (0)','% Abandona (1)']
    display(tab)
    ax = tab.plot(kind='bar', figsize=(8,4))
    ax.set_title(f'Abandono por rangos de {col}')
    ax.set_ylabel('Porcentaje')
    ax.set_xlabel('Rango')
    plt.tight_layout()
    plt.show()

for c in ['meses_en_empresa','cargos_mensuales','cargos_totales','cuentas_diarias']:
    if c in telecom_df.columns:
        churn_por_bins(telecom_df, c, q=5)


##  Mapa de calor de correlaciones (numéricas)


In [ ]:
num_cols = telecom_df.select_dtypes(include=[np.number]).columns
corr = telecom_df[num_cols].corr()

plt.figure(figsize=(10,6))
plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(label='Correlación')
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.title('Mapa de calor de correlaciones (numéricas)')
plt.tight_layout()
plt.show()

# Vista de correlaciones contra la variable objetivo (si existe)
if 'abandono' in corr.columns:
    display(corr['abandono'].sort_values(ascending=False))


##  Conclusiones 

- **Contrato mensual** suele concentrar mayor churn; los contratos más largos reducen la evasión.  
- **Clientes nuevos** (baja tenencia) abandonan más → priorizar retención temprana.  
- **Cargos mensuales altos** incrementan el riesgo si no hay valor agregado percibido.  
- **Electronic check** suele asociarse con mayor churn vs pagos automáticos.  
- **Servicios y engagement** (soporte, seguridad) tienden a disminuir la evasión.

## Recomendaciones

1. **Onboarding 0–6 meses** con contacto proactivo y beneficios.  
2. **Migración a contratos anual/bianual** con incentivos.  
3. **Revisión de planes de alto cargo** (bundles de valor/downgrade acompañados).  
4. **Fomentar pagos automáticos** (tarjeta/transferencia).  
5. **Campañas preventivas** para perfiles de riesgo combinando señales (contrato mensual + baja tenencia + electronic check + cargos altos).
